In [14]:
import tensorflow as tf
import os
import subprocess
import cv2
import pandas as pd
from matplotlib import pyplot as plt

In [15]:
datapath = './Dataset/'
dataset = os.listdir(datapath)

print(dataset)

['Test', 'Test.txt', 'Train', 'Train.txt', 'Validation', 'Validation.txt']


In [42]:
train_labels = pd.read_csv('./Labels/TrainLabels.csv')
test_labels = pd.read_csv('./Labels/TestLabels.csv')
val_labels = pd.read_csv('./Labels/ValidationLabels.csv')

In [45]:
# Adapted from extractFrames.py
x_train, x_val, x_test = [], [], []
y_train, y_val, y_test = [], [], []

for ttv in dataset:
    print("Extracting", ttv, "...")
    if ttv[-4:] == '.txt':
        continue
    users = os.listdir(datapath + ttv + '/')
    for user in users:
        currUser = os.listdir(datapath + ttv + '/' + user + '/')
        for extract in currUser:
            clip = os.listdir(datapath + ttv + '/' + user + '/' + extract + '/')[0]
            path = datapath + ttv + '/' + user + '/' + extract + '/'
            vidcap = cv2.VideoCapture(path + clip)
            
            frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
            success, image = vidcap.read()
            count = 1
            while success and count < frame_count / 2:
                success, image = vidcap.read()
                count += 1
            
#             plt.imshow(image, interpolation='nearest')
            pairing = (clip, image)
            if ttv == 'Train':
                label = train_labels.loc[train_labels['ClipID'] == clip]
                x_train.append(pairing)
                y_train.append(label)
            elif ttv == 'Validation':
                label = val_labels.loc[train_labels['ClipID'] == clip]
                x_val.append(pairing)
                y_val.append(image)
            elif ttv == 'Test':
                label = test_labels.loc[train_labels['ClipID'] == clip]
                x_test.append(pairing)
                y_test.append(image)

Extracting Test ...
Extracting Test.txt ...
Extracting Train ...
Extracting Train.txt ...
Extracting Validation ...
Extracting Validation.txt ...


In [54]:
for pairing in x_train:
    clip_name, image = pairing[0], pairing[1]
    cv2.imwrite('./MidFrameDataSet/Train/' + clip_name[:-4] + '.jpg', image)

In [55]:
for pairing in x_val:
    clip_name, image = pairing[0], pairing[1]
    cv2.imwrite('./MidFrameDataSet/Validation/' + clip_name[:-4] + '.jpg', image)

In [56]:
for pairing in x_test:
    clip_name, image = pairing[0], pairing[1]
    cv2.imwrite('./MidFrameDataSet/Test/' + clip_name[:-4] + '.jpg', image)

In [58]:
all_labels = pd.read_csv('./Labels/AllLabels.csv')
all_labels

,ClipID,Boredom,Engagement,Confusion,Frustration
0,1100011002.avi,0,2,0,0
1,1100011003.avi,0,2,0,0
2,1100011004.avi,0,3,0,0
3,1100011005.avi,0,3,0,0
4,1100011006.avi,0,3,0,0
...,...,...,...,...,...
8920,9877360164.avi,1,3,0,0
8921,9877360165.avi,0,3,0,0
8922,9877360166.avi,1,3,0,2
8923,9877360168.avi,1,3,1,1


In [62]:
all_labels['ImageID'] = all_labels['ClipID'].apply(lambda x: x[:-4] + '.jpg')
all_labels

,ClipID,Boredom,Engagement,Confusion,Frustration,ImageID
0,1100011002.avi,0,2,0,0,1100011002.jpg
1,1100011003.avi,0,2,0,0,1100011003.jpg
2,1100011004.avi,0,3,0,0,1100011004.jpg
3,1100011005.avi,0,3,0,0,1100011005.jpg
4,1100011006.avi,0,3,0,0,1100011006.jpg
...,...,...,...,...,...,...
8920,9877360164.avi,1,3,0,0,9877360164.jpg
8921,9877360165.avi,0,3,0,0,9877360165.jpg
8922,9877360166.avi,1,3,0,2,9877360166.jpg
8923,9877360168.avi,1,3,1,1,9877360168.jpg


In [65]:
all_labels.drop('ClipID', inplace=True, axis=1)

KeyError: "['ClipID'] not found in axis"

In [67]:
all_labels.to_csv('./MidFrameDataSet/labels.csv')